# Anomaly Detection | GPT-4 Vision

## GPT-4 Vision

GPT-4 Vision, with its advanced capability to interpret and analyze images, can be employed to detect anomalies in data. By plotting the time series data into a graph, the visual representation can be provided to the GPT-4 Vision model, which can then identify irregular patterns and deviations from expected behavior.

The [reference](../../TestData/TestData_Reference.txt) time series data and the time series data with [anomalies](../../TestData/TestData_Degradation.txt) introduced in [how to use embeddings to detect anomalies](./AnomalyDetection-Embedding.ipynb) has been visualized in a simple diagram: 


![](../../media/img/AnomalyChart.png)



## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed. ([Details and instructions can be found here](../../CreateEnv/CreateEnv.azcli))


## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. 

In [24]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.17"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string openAIApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY") ?? "";
string openAIEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") ?? "";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "";
string completionDeploymentName = Environment.GetEnvironmentVariable("COMPLETION_DEPLOYMENTNAME") ?? "";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(openAIApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(openAIEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Client created...");

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.17 DotNetEnv, 2.5.0

OpenAI Client created...


## Step2: Create Visualization

Create time series data visualization

In [25]:
using System.Drawing; 
using System.Drawing.Imaging;
using System.Text.Json;
using System.IO;

#!import CreateDiagram.cs

public class DeviceData
{
    public string DeviceId { get; set; } = "";
    public string TimeStamp { get; set; } = "";
    public float EnergyConsumption { get; set; }
    public float Pressure { get; set; }
}

string[] referenceData = await File.ReadAllLinesAsync("../../TestData/TestData_Reference.txt");
string[] degradationData = await File.ReadAllLinesAsync("../../TestData/TestData_Degradation.txt");


PointF[] pointsReference = new PointF[10];
int i=0;
foreach(string line in referenceData)
{    
    DeviceData deviceDataReference = JsonSerializer.Deserialize<DeviceData>(line)?? new DeviceData();
    pointsReference[i++] = new PointF(i*100, deviceDataReference.Pressure * 100);
}
Console.WriteLine($"Reference data points created...");

PointF[] pointsDegradation = new PointF[10];
i=0;
foreach(string line in degradationData)
{
    DeviceData deviceDataDegradation = JsonSerializer.Deserialize<DeviceData>(line)?? new DeviceData();
    pointsDegradation[i++] = new PointF(i*100, deviceDataDegradation.Pressure * 100);
}
Console.WriteLine($"Degradation data points created...");

string diagramFile = "./TestDiagram.jpg";
await Tool.CreateDiagram(diagramFile, 1000, 1000, pointsReference, pointsDegradation);
Console.WriteLine($"Diagram created...");

Reference data points created...
Degradation data points created...
Diagram created...


## Step 3: Call GPT-4 Vision

GPT-4 Vision is called with a system prompt instructing the model to look for anomalies. The previously created time series diagram is provided as a byte array.


In [26]:

string systemPrompt = @"
    You analyze time series data shown in a x,y diagram. 
    You always get two graphs presented in one diagram. 
    You see pressure values from a compressor. 
    You analyze if the pressure is equal in both graphs over a specific period of time. 
    The time is shown in the x-axis. 
    You are sensitive and check if the top value (y-axis) of both graphs is equal over the shown time period. 
    You answer with a valid JSON string. 
    The format of the JSON string is {""Anomaly"": ""yes""} if the pressure value is not equal over time. 
    You answer with {""Anomaly"": ""no""} if the pressure value is equal over time.
";
string prompt = "Given the diagram above, can you detect any anomaly in the pressure data?";

BinaryData binaryData = BinaryData.FromBytes(await File.ReadAllBytesAsync(diagramFile));
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = completionDeploymentName,
    Messages =
    {
        new ChatRequestSystemMessage(systemPrompt),
        new ChatRequestUserMessage(prompt),
        new ChatRequestUserMessage(
            new ChatMessageTextContentItem(prompt),
            new ChatMessageImageContentItem(binaryData, "image/jpeg")
        )
    },
    MaxTokens = 100, 
    NucleusSamplingFactor = 0.1f,
    Temperature = 0.1f,
};

Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);
Console.WriteLine($"{response.Value.Choices[0].Message.Content}");


{"Anomaly": "yes"}
